In [1]:
import torch 
from torch.utils.data import DataLoader, Dataset
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path
from tqdm.notebook import tqdm
from albumentations.pytorch import ToTensorV2
import albumentations as A
import matplotlib.pyplot as plt
import glob, matplotlib.pyplot as plt, matplotlib.patches as patches

In [3]:
train_path = '../../Data/train_images/'
train_ans_path = '../../Data/train_anns/'
test_path = '../../Data/test_images/'
sub_sample_path = '../../Data/detection-results/'

train_img_ls = os.listdir(train_path)
print(len(train_img_ls))

test_im_ls = os.listdir(test_path)
len(test_im_ls)

4000


1000

In [4]:
## Display the ans
train_ans_ls = os.listdir(train_ans_path)  # txt

def Load_ans(txt_path):
    
    
    with open(txt_path, 'r') as F:
        rows = F.readlines()
        F.close()
    train_ans = []
    for row in tqdm(rows):
        train_ans.append(row.strip('\n'))
        
    return train_ans

In [5]:
train_ans_ls.sort()
train_img_ls.sort()
train_df = pd.DataFrame({
    'img_path': train_img_ls,
    'label_path': train_ans_ls[1:]
})

# rain_df.to_csv('../train_df.csv')

In [6]:
CONFIG = {
    'batch_size': 8,
    'lr': 1e-3,
    'num_workers': 8,
    'epoch': 20
}
CLASS_NAME = ['helmet', 'head', 'person']
NUM_CLASSES = len(CLASS_NAME)
NUM_CLASSES

3

In [6]:
## Convert str to int
def str2int(inp_list):
    return list(map(int, inp_list))

def Convert_label2xml(txt_path, target_path=None):
    '''
    将txt格式的label 转化成xml
    args:
        txt_path: Path of the txt
        target_path: Path to save new format
    '''
    label_dict = {}
    bboxes = []
    with open(txt_path, 'r') as F:
        rows = F.readlines()
        F.close()
    # print(rows)
    num = len(rows)
    head_id = []
    for i, row in enumerate(rows):
        row = row.strip('\n')
        # print(row)
        # print(row.split(' ')[0])
        if row.split(' ')[0] == 'head':
            head_id.append(i)
        bbox = row.split(' ')[1:]
        ## 转化为 int
        bboxes.append(str2int(bbox))
    # print(bboxes)
    # print(head_id)
    label_dict['boxes'] = torch.from_numpy(np.array(bboxes))
    labels = torch.ones(num) 
    labels[head_id] = 0
    label_dict['labels'] = labels
    return label_dict

In [7]:
## dataset
class helmet_dataset(Dataset):
    def __init__(self, df, class_name = CLASS_NAME, train_ans_path='../../Data/train_anns/', train_img_path='../../Data/train_images/', test_img_path='../../Data/test_images/', transforms=None, is_train=True):
        self.df = df
        self.train_ans_path = train_ans_path
        self.train_img_path = train_img_path
        self.test_img_path = test_img_path
        self.class_name = class_name
        self.is_train = is_train
        if transforms:
            self.transforms= transforms
        else:
            self.transforms = A.Compose([
                A.Resize(415, 415),
                ToTensorV2()
            ])
    def __getitem__(self ,idx):
        if self.is_train:
            row = self.df.iloc[idx]
            img = cv2.imread(os.path.join(self.train_img_path, row['img_path']))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.
            img = self.transforms(image=img)['image']
            label = self.Convert_label2xml(os.path.join(self.train_ans_path, row['label_path'])) ## dict

            return img, label
        else:
            row = self.df.iloc[idx]
            img = cv2.imread(os.path.join(self.test_img_path, row['img_path']))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.
            img = self.transforms(image=img)['image']
            
            return img
    def __len__(self):
        return len(self.df)

    def str2int(self, inp_list):
        return list(map(int, inp_list))

    def Convert_label2xml(self, txt_path, target_path=None):
        '''
        将txt格式的label 转化成xml
        args:
            txt_path: Path of the txt
            target_path: Path to save new format
        '''
        label_dict = {}
        bboxes = []
        with open(txt_path, 'r') as F:
            rows = F.readlines()
            F.close()
        # print(rows)
        num = len(rows)
        head_id = []
        # bg_id =[]
        person_id = []
        for i, row in enumerate(rows):
            row = row.strip('\n')
            # print(row)
            # print(row.split(' ')[0])
            if row.split(' ')[0] == 'head':
                head_id.append(i)
            if row.split(' ')[0] == 'person':
                person_id.append(i)
            # if row.split(' ')[0] == '__background':
            #     bg_id.append(i)
            bbox = row.split(' ')[1:]
            ## 转化为 int
            bboxes.append(self.str2int(bbox))
        # print(bboxes)
        # print(head_id)
        label_dict['boxes'] = torch.from_numpy(np.array(bboxes))
        labels = torch.zeros(num)   ## helmet 0
        labels[head_id] = 1         ## head 1
        # labels[bg_id] = 3         ## bg   3  
        labels[person_id] = 2       ## person 2
        label_dict['labels'] = labels
        return label_dict
    # def finnal_label(self, kwargs):
    #     '''
    #     args:
    #         kawargs: -->dict {'labels': , 'boxes': }
    #     '''
    #     boxex, labels = [], []
    #     for lbl in kwargs['labels']:
    #         labels.append(self.class_name.index())
    
# The collate_fn() will help us take care of tensors of varying sizes while creating
# the training and validation data loaders.
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number 
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

In [10]:
# creating customised FasterRCNN model

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, MultiScaleRoIAlign

def create_model(num_classes):
    # load Faster RCNN pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get the number of input features 
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    ## define a new box_roi_pool
    model.roi_heads.box_roi_pool = MultiScaleRoIAlign(featmap_names=['0', '1', '2'], output_size=(7,7), sampling_ratio=2)
    # define a new head for the detector with required number of classes
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    return model

In [11]:
# initialize the model and move to the computation device GPU
model = create_model(num_classes=NUM_CLASSES)
model = model.to('cuda')
ckpt = torch.load('../../ckpts/9_30/via_train/best_model.pth')
model.load_state_dict(ckpt['model'])
print('Training_loss :{}'.format(ckpt['loss']))

Training_loss :[2.4561749779715827, 0.25957817176931225]


In [68]:
# test_data
test_im_ls = os.listdir(test_path)
len(test_im_ls)
test_im_ls.sort()
test_df = pd.DataFrame({
    'img_path': test_im_ls
})
test_df

test_set = helmet_dataset(test_df, is_train=False)

In [188]:
def write2txt(label, bbox, scores, target_path):
    
    with open(target_path, 'w') as f:
        for i in range(len(label)):
            # print(i)
            f.write(CLASS_NAME[label[i]])
            f.write(' ' + str(scores[i]) + ' ')
            f.write(str(bbox[i][0]) + ' ')
            f.write(str(bbox[i][1]) + ' ')
            f.write(str(bbox[i][2]) + ' ')
            f.write(str(bbox[i][3]) + ' ')
            f.write('\n')

In [190]:
## inference
model.eval()
# test_pbar = tqdm(test_set, total=len(test_set))
# label = []
# bbox = []
with torch.no_grad():
    for data, name in tqdm(test_set):
        out = model(data.cuda().to(torch.float32).unsqueeze(0))  ## return dict
        label = list(out[0]['labels'].detach().cpu().numpy())
        bbox = list(out[0]['boxes'].detach().cpu().numpy())
        score = list(out[0]['scores'].detach().cpu().numpy())
        ## write
        
        write2txt(label, bbox, score, target_path='../res/9_29/'+name.replace('png', 'txt'))
        

0it [00:00, ?it/s]

In [187]:
out

[{'boxes': tensor([[ 37.3901, 162.6664, 120.1916, 260.4206],
          [134.6002, 192.7815, 149.4224, 211.1556]], device='cuda:0'),
  'labels': tensor([1, 1], device='cuda:0'),
  'scores': tensor([0.9995, 0.1025], device='cuda:0')}]

In [149]:
label = list(out[0]['labels'].detach().cpu().numpy())

bbox = list(out[0]['boxes'].detach().cpu().numpy())

In [21]:
## 验证submit
sample_path = '../Data/detection-results/'
sub_path = '../res/9_29/'

assert len(os.listdir(sample_path)) == len(os.listdir(sub_path))

samples = os.listdir(sample_path).sort()
submits = os.listdir(sub_path).sort()



In [22]:
samples = os.listdir(sample_path)
submits = os.listdir(sub_path)

In [12]:
submits.remove('.ipynb_checkpoints')

In [23]:
len(submits)

1001

In [18]:
os.removedirs('../res/9_29/.ipynb_checkpoints', )

OSError: [Errno 39] Directory not empty: '../res/9_29/.ipynb_checkpoints'

In [20]:
%%script

!rm -rf ../res/9_29/.ipynb_checkpoints/

IndexError: list index out of range

In [24]:
!curl -L -o /usr/bin/arc http://autodl-public.ks3-cn-beijing.ksyun.com/tool/arc && chmod +x /usr/bin/arc
!arc compress detection-results.zip ../res/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3071k  100 3071k    0     0  5838k      0 --:--:-- --:--:-- --:--:-- 5838k
